In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

from tqdm.notebook import tqdm

In [37]:
class Layer():
    
    #Activation Functions and their derivatives
    @staticmethod
    def sigmoid(x):
        return np.where(x >= 0, 1. / (1. + np.exp(-x)), np.exp(x) / (1. + np.exp(x)))
    
    @staticmethod
    def dsigmoid(x):
        return Layer.sigmoid(x) * (1. - Layer.sigmoid(x))  
    
    @staticmethod
    def softmax(x):
        exp = np.exp(x - np.max(x))
        return_vector = exp / np.sum(exp)
        return_vector[return_vector==0] = 10 ** -9
        return return_vector
    
    @staticmethod
    def dsoftmax(x):
        return Layer.softmax(x) * (1 - Layer.softmax(x))
        
    
    @staticmethod
    def relu(x):
        return np.where(x>=0, x, 0)
    
    @staticmethod
    def drelu(x):
        return np.where(x>=0, 1, 0)
    
    @staticmethod
    def leaky_relu(x):
        return np.where(x>=0, x, 0.01*x)
        
    @staticmethod
    def dleaky_relu(x):
        return np.where(x>=0, 1, 0.01)
        
    functions = {
        'sigmoid': sigmoid.__func__,
        'softmax': softmax.__func__,
        'relu': relu.__func__,
        'leaky_relu': leaky_relu.__func__,
        'input': np.nan
    }
    
    dfunctions = {
        'sigmoid': dsigmoid.__func__,
        'softmax': dsoftmax.__func__,
        'relu': drelu.__func__,
        'leaky_relu': dleaky_relu.__func__,
        'input': np.nan
    }
    
    #ALL INITIALIZERS ARE NORMAL
    @staticmethod
    def golrot(n_nodes, n_prev):
        return np.sqrt(2 / (n_nodes + n_prev))
    
    @staticmethod
    def he(n_nodes, n_prev):
        return np.sqrt(2 / n_prev)
    
    @staticmethod
    def lecun(n_nodes, n_prev):
        return np.sqrt(1 / n_prev)
    
    initializers = {
        'golrot': golrot.__func__,
        'he': he.__func__,
        'lecun': lecun.__func__
    }
    
    def __init__(self, n_nodes, initializer='golrot', activation='sigmoid'):
        self.n_nodes = n_nodes
        self.initializer = Layer.initializers[initializer]
        self.activation = Layer.functions[activation]
        self.dactivation = Layer.dfunctions[activation]

class NeuralNetwork():
    
    #Cost functions and their derivatives
    @staticmethod
    def quadratic(y, pred):
        return np.sum((y - pred) ** 2) * 0.5
    
    @staticmethod
    def dquadratic(y, pred):
        return pred - y
    
    @staticmethod
    def cross_entropy(y, pred):
        return -1. * np.dot(y, np.log(pred)) / len(y)
    
    @staticmethod
    def dcross_entropy(y, pred):
        return pred - y
    
    costs = {
        'quadratic': quadratic.__func__,
        'log_loss': cross_entropy.__func__,
        'cross_entropy': cross_entropy.__func__
    }
    
    dcosts = {
        'quadratic': dquadratic.__func__,
        'log_loss': dcross_entropy.__func__,
        'cross_entropy': dcross_entropy.__func__
    }

    def __init__(self, cost, lr=0.03, epochs=10, verbose=True):
        self.lr = lr
        self.epochs = epochs
        self.verbose = verbose
        self.cost = NeuralNetwork.costs[cost]
        self.dcost = NeuralNetwork.dcosts[cost]
        
    def fit(self, X, y, layers):
        self.X = X
        self.y = y
        
        #Input layer
        self.layers = []
        self.layers.append(Layer(self.X[0].shape[0], activation='input'))
        
        #Initialize layers and weights
        for index, layer in enumerate(layers):                                               
            layer.n_prev = self.layers[index].n_nodes
            
            layer.weights = np.random.normal(scale=layer.initializer(layer.n_nodes, layer.n_prev),
                                             size=(layer.n_nodes, layer.n_prev))
            
            self.layers.append(layer)
            

        for epoch in range(self.epochs):
            if self.verbose==True:
                print(f'Epoch: {epoch+1}')
            epoch_cost = 0
            
            for X, y in tqdm(zip(self.X, self.y), total=self.X.shape[0]):
                self.layers[0].a = X
                
                #Propagation
                for layer in self.layers[1:]:
                    layer.z = np.dot(layer.weights, self.layers[self.layers.index(layer)-1].a)
                    layer.a = layer.activation(layer.z)
                
                #Errors
                epoch_cost += self.cost(y, self.layers[-1].a)
                self.layers[-1].error = np.multiply(self.dcost(y, self.layers[-1].a),
                                        self.layers[-1].dactivation(self.layers[-1].z))
                
                for layer in reversed(self.layers[1:-1]):
                    layer.error = (np.dot(np.transpose(self.layers[self.layers.index(layer)+1].weights),
                            self.layers[self.layers.index(layer)+1].error)) * layer.dactivation(layer.z)
                    
                    
                #Gradients and update weights
                for layer in self.layers[1:]:
                    layer.gradients = np.dot(layer.error.reshape(-1, 1),
                                      self.layers[self.layers.index(layer)-1].a.reshape(1, -1))
                    
                    layer.weights -= self.lr * layer.gradients
                    
            
            #ADD METRICS
            if self.verbose==True:
                print(f'Cost: {(epoch_cost / self.y.shape[0]).round(4)}')
                
    
    def predict(self, X):
        self.X_test = X
        
        predictions = []
        for X in self.X_test:
            #Input
            self.layers[0].a = X
            
            #Propagation
            for layer in self.layers[1:]:
                layer.z = np.dot(layer.weights, self.layers[self.layers.index(layer)-1].a)
                layer.a = layer.activation(layer.z)
                
            predictions.append(np.where(self.layers[-1].a>0.5, 1, 0))
                
        return predictions

In [70]:
X, y = make_classification(n_samples=5000, n_features=20, n_informative=20, n_redundant=0, n_classes=2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

nn = NeuralNetwork(lr=0.3, cost='quadratic', epochs=5, verbose=True)

layers = [
    Layer(20),
    Layer(10),
    Layer(1)
]

nn.fit(X_train, y_train, layers)
pred = nn.predict(X_test)

print(f'Prediction Accuracy: {accuracy_score(y_test, pred)*100}%')

Epoch: 1



Cost: 0.0701
Epoch: 2




Cost: 0.0441
Epoch: 3



Cost: 0.0352
Epoch: 4



Cost: 0.0293
Epoch: 5



Cost: 0.0239
Prediction Accuracy: 90.10000000000001%


In [90]:
X, y = make_classification(n_samples=10000, n_features=30, n_informative=30, n_redundant=0, n_classes=10)
y_classes = np.copy(y)
y = np.array(pd.get_dummies(y))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

nn = NeuralNetwork(lr=0.003, cost='log_loss', epochs=60, verbose=True)

layers = [
    Layer(100, activation='leaky_relu'),
    Layer(100, activation='leaky_relu'),
    Layer(100, activation='leaky_relu'),
    Layer(10, activation='softmax')
]

nn.fit(X_train, y_train, layers)
pred = nn.predict(X_test)

print(f'Prediction Accuracy: {accuracy_score(y_test, pred)*100}%')

Epoch: 1





Cost: 0.2173
Epoch: 2



Cost: 0.1757
Epoch: 3



Cost: 0.1502
Epoch: 4



Cost: 0.1311
Epoch: 5


...


Cost: 0.0218
Epoch: 57



Cost: 0.0231
Epoch: 58



Cost: 0.0225
Epoch: 59



Cost: 0.0219
Epoch: 60



Cost: 0.0226
Prediction Accuracy: 75.8%
